In [ ]:
# required packages
import datetime as dt
from sklearn.cluster import KMeans

In [ ]:
# Define snapshot date
snapshot_date = df['InvoiceDate'].max() + dt.timedelta(days=1)

In [ ]:
# Aggregate data on a customer level
customers = df.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

In [ ]:
# Rename columns
customers.rename(columns = {'InvoiceDate': 'Recency',
                            'InvoiceNo': 'Frequency',
                            'TotalSum': 'MonetaryValue'}, inplace=True)

In [ ]:
# Log transformation
customers_log = np.log(customers)

In [ ]:
# Perform clustering
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(customers_log)